In [1]:
pip install transformers

     |████████████████████████████████| 2.6 MB 4.9 MB/s 
     |████████████████████████████████| 636 kB 43.4 MB/s 
     |████████████████████████████████| 895 kB 48.1 MB/s 
     |████████████████████████████████| 3.3 MB 45.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")

model = AutoModelForSequenceClassification.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/920 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
xls = pd.ExcelFile('Негатив_пример.xlsx')
xls_toxic = pd.read_excel('Негатив_пример.xlsx', index_col=None, header=0)  

In [ ]:
xls_toxic['Text'][3].split(' ')

['завтра', 'же', 'нахер', 'все', 'позакрою', 'тут,', 'ебучее', 'говно']

In [ ]:
def razbienie(str):
    spisok_str = str.split(' ')

    new_spisok = []
    i = 0
    if len(spisok_str) == 1:
        new_spisok.append(spisok_str[i])
        return new_spisok
    if len(spisok_str) % 2 != 0:
        while i <= len(spisok_str)-4:
            new_spisok.append(spisok_str[i] + ' ' + spisok_str[i+1])
            i += 2
        new_spisok.append(spisok_str[i] + ' ' + spisok_str[i+1] + ' ' + spisok_str[i+2])
    else:
        while i <= len(spisok_str)-2:
            new_spisok.append(spisok_str[i] + ' ' + spisok_str[i+1])
            i += 2
    return new_spisok

In [ ]:
pt_batches = []
for i in range(len(xls_toxic['Text'])):
  #print(xls_toxic['Text'][i].split(' '))
  pt_batch = tokenizer(
    razbienie(xls_toxic['Text'][i]),
    padding=True,    truncation=True,    max_length=512,    return_tensors="pt" )
  pt_batches.append(pt_batch)

In [ ]:
from torch import nn
pt_predictions = []
for pt_batch in pt_batches:
  pt_output = model(**pt_batch)
  pt_prediction = nn.functional.softmax(pt_output.logits, dim=-1)
  pt_predictions.append(pt_prediction)

In [ ]:
pt_predictions[5]

tensor([[0.0545, 0.9455]], grad_fn=<SoftmaxBackward>)

In [ ]:
xls_toxic['Text'][3], pt_predictions[3]

('завтра же нахер все позакрою тут, ебучее говно', tensor([[0.2170, 0.7830],
         [0.0884, 0.9116],
         [0.9681, 0.0319],
         [0.0412, 0.9588]], grad_fn=<SoftmaxBackward>))

In [ ]:
import numpy as np
bad_words_full = []
procents = []
for i in range(len(pt_predictions)):
  bad_words = []
  words = razbienie(xls_toxic['Text'][i])
  #print(float(pt_predictions[i][0][1])) 
  for j in range(len(pt_predictions[i])):
    if float(pt_predictions[i][j][1]) > 0.83:
      #print(type(words[j]))
      bad_words.append((words[j], np.round(float(pt_predictions[i][j][1]), 4)))
      #bad_words.append(float(pt_predictions[i][j][1]))
  procent = len(bad_words)/len(words)
  procents.append(procent)
  bad_words_full.append(bad_words)
print(bad_words_full)

[[], [], [('деньги паскуды,', 0.9572)], [('нахер все', 0.9116), ('ебучее говно', 0.9588)], [], [('Говно', 0.9455)], [('за хрень', 0.9814)], [], [('буду писать жалобу', 0.881)], [('Вы издеваетесь', 0.9293), ('я не', 0.92)], [], [], [('вы издеваетесь?', 0.9571)], [], [], [('Пидары ебаные !!!', 0.9927)], [], [], [], [('Уроды вы ебаные', 0.9807)], [], [], [('я их', 0.9876), ('за херня происходит', 0.9868)], [('Вы издевается?', 0.9717)], [], [], [('за херня', 0.9926), ('опять у вас', 0.8513)], [], [('Что за херня', 0.9945)], [('блять ну', 0.9692), ('хуйня ?????', 0.9855)], [('Ало блять', 0.9679)], [], [], [], [], [], [('издеваетесь ', 0.9586)], [], [('вы издеваетесь', 0.9293)], [('Вы издеваетесь?', 0.9571)], [('я должна', 0.9157)], [('Помоему вы издеваетесь', 0.962)], [], [], [('че блядь', 0.9872)], [('самая ебанутая поддержка', 0.9652)], [('Вы издеваетесь', 0.9293)], [('Издеваетесь? Вы', 0.9313)], [], [], [], [], [], [('со своим', 0.8577), ('тупым приложением,', 0.9579), ('часа пялиться', 

In [ ]:
xls_toxic['Bad_words'] = bad_words_full
xls_toxic['Percentage of bad'] = np.round(procents, 4)

In [ ]:
xls_toxic

,Text,Bad_words,Percentage of bad
0,ужасная поддержка!,[],0.00
1,ужасная поддержка,[],0.00
2,"где мой деньги паскуды, сколько можно херней з...","[(деньги паскуды,, 0.9572)]",0.25
3,"завтра же нахер все позакрою тут, ебучее говно","[(нахер все, 0.9116), (ебучее говно, 0.9588)]",0.50
4,У вас приложение говно полное,[],0.00
...,...,...,...
105,"скажите, где и кому сообщать по поводу работы ...",[],0.00
106,что за хуйня опять у вас там творится,"[(хуйня опять, 0.9936)]",0.25
107,потому по щётку такая хуйня,"[(щётку такая хуйня, 0.9947)]",0.50
108,"что значит ""зафиксировать обращение""? я хочу в...",[],0.00


In [ ]:
xls_toxic.to_excel('негатив_вывод_ver_4.xlsx')

In [9]:
import os
import pandas as pd
from torch import nn
import numpy as np

a = os.listdir(path='./xlsx')
count = 0
for name_xlsx_in_dir in a: 
  if name_xlsx_in_dir.endswith('.xlsx') == True:
    path = './xlsx/' + name_xlsx_in_dir
#    print(path)   
    xls = pd.ExcelFile(path)
    xls_toxic = pd.read_excel(path, index_col=None, header=0) #, skiprows=3

# -//-

    def razbienie(str):
        spisok_str = str.split(' ')
        new_spisok = []
        i = 0
        if len(spisok_str) == 1:
            new_spisok.append(spisok_str[i])
            return new_spisok
        if len(spisok_str) % 2 != 0:
            while i <= len(spisok_str)-4:
                new_spisok.append(spisok_str[i] + ' ' + spisok_str[i+1])
                i += 2
            new_spisok.append(spisok_str[i] + ' ' + spisok_str[i+1] + ' ' + spisok_str[i+2])
        else:
            while i <= len(spisok_str)-2:
                new_spisok.append(spisok_str[i] + ' ' + spisok_str[i+1])
                i += 2
        return new_spisok

# -//-

    pt_batches = []
    for i in range(len(xls_toxic['Text'])):
      #print(xls_toxic['Text'][i].split(' '))
      pt_batch = tokenizer(
        razbienie(str(xls_toxic['Text'][i])),
        padding=True,    truncation=True,    max_length=512,    return_tensors="pt" )
      pt_batches.append(pt_batch)

    pt_predictions = []
    for pt_batch in pt_batches:
      pt_output = model(**pt_batch)
      pt_prediction = nn.functional.softmax(pt_output.logits, dim=-1)
      pt_predictions.append(pt_prediction)

# -//-

    bad_words_full = []
    procents = []
    for i in range(len(pt_predictions)):
      bad_words = []
      words = razbienie(str(xls_toxic['Text'][i]))
      #print(float(pt_predictions[i][0][1])) 
      for j in range(len(pt_predictions[i])):
        if float(pt_predictions[i][j][1]) > 0.83:
          #print(type(words[j]))
          bad_words.append((words[j], np.round(float(pt_predictions[i][j][1]), 4)))
          #bad_words.append(float(pt_predictions[i][j][1]))
      procent = len(bad_words)/len(words)
      procents.append(procent)
      bad_words_full.append(bad_words)
    #print(bad_words_full)

# -//-

    xls_toxic['Bad_words'] = bad_words_full
    xls_toxic['Percentage of bad'] = np.round(procents, 4)

# -//-

    path_exit = './Выводы/Негатив_вывод_v4(p2)_'+ str(count) + '.xlsx'
    xls_toxic.to_excel(path_exit)
    count += 1
    print(count)



1
2
3
4
5
6
7
8
9
10
